In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import ChainMap
from collections import Counter
from itertools import combinations

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1046,tt1931533,15000000,19422261,Seven Psychopaths,Colin Farrell|Sam Rockwell|Christopher Walken|...,Martin McDonagh,They Won't Take Any Shih Tzu.,A struggling screenwriter inadvertently become...,110,Comedy|Crime|Thriller,Blueprint Pictures|CBS Films|British Film Inst...,9/7/2012,6.6,2012
782,tt0810913,79000000,149673788,Jack and Jill,Adam Sandler|Katie Holmes|Al Pacino|Eugenio De...,Dennis Dugan,His twin sister is coming for the holidays... ...,"Jack Sadelstein, a successful advertising exec...",91,Comedy,Columbia Pictures|Happy Madison Productions|Br...,11/11/2011,4.2,2011
268,tt0380510,65000000,93525586,The Lovely Bones,Rachel Weisz|Mark Wahlberg|Susan Sarandon|Saoi...,Peter Jackson,The story of a life and everything that came a...,"After being brutally murdered, 14-year-old Sus...",136,Fantasy|Drama,WingNut Films|DreamWorks SKG|Key Creatives|Fil...,12/26/2009,6.6,2009
1540,tt0404390,15000000,9500000,Running Scared,Paul Walker|Cameron Bright|Vera Farmiga|Chazz ...,Wayne Kramer,Every bullet leaves a trail.,"After a drug-op gone bad, Joey Gazelle is put ...",122,Action|Crime|Drama|Thriller,International Production Company|True Grit Pro...,1/6/2006,6.8,2006
823,tt1637706,10000000,17273593,Our Idiot Brother,Paul Rudd|Elizabeth Banks|Zooey Deschanel|Emil...,Jesse Peretz,Everybody has one.,Everybody has one. The sibling who is always j...,90,Comedy|Drama,The Weinstein Company,8/26/2011,5.7,2011


In [670]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [671]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [3]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
data.release_date = pd.to_datetime(data.release_date)

# profit = revenue - budget
data['profit'] = data.revenue - data.budget

def str_split(string):
    ''' The function create a list from a string'''
    return string.split('|')


def list_creator(series):
    ''' The function takes Pandas.Series
        then create a list from a string
        then create collections.Counter 
        and return it '''
    return Counter([unit for string in series for unit in string.split('|')])


def couple_creator(string):
    ''' The function create a list from a string,
        then create actor's couples from the list'''
    cast_list = string.split('|')
    couple = []
    for actor in cast_list:
        for name in range(len(cast_list)):
            if actor > cast_list[name]:
                couple.append( cast_list[name] +' & ' + actor)
    
    return couple


def couple_creator_1(string):
    ''' The function create a list from a string,
        then create actor's couples from the list'''
    cast_list = string.split('|')
    
    return [actor +' & ' + cast_list[name] for actor in cast_list\
              for name in range(len(cast_list)) if cast_list[name] > actor]


def couple_creator_2(string):
    ''' The function create a list from a string
        and create a list of actor's couples from the list'''
    return list(combinations(string.split('|'), 2))


def couple_creator_3(series):
    '''The function takes a Pandas.Series
       then create а list of couples from a Series.string
       and return the most common couples'''
    return Counter([couple for couple_list in series.apply(couple_creator_1)\
                    for couple in couple_list]).most_common(10)


#  The function with use the itertools.combinations()   
def dict_creator(series):
    ''' The function a Pandas.Series
        then create a list from a Series.string
        then create a pairs dictionary(couple_dict) from the list
        then create a pairs dictionary with max values only(couple_max)
        and return the couple_max'''
    couple_dict = Counter()
    for string in series:
        string = couple_creator_2(string)
        for actor in string:
            couple_dict[actor] += 1
            
    max_number = max(couple_dict.values())
    couple_max = {}              
    for couple in couple_dict:
        if couple_dict[couple] == max_number:
            couple_max[couple] = couple_dict[couple] 
            
    return couple_max 


#  The function without use the itertools.combinations() 
def dict_creator_1(series):
    ''' The function takes a Pandas.Series
        then create a list from a Series.string
        then create a pairs dictionary(couple_dict) from the list
        then create a pairs dictionary with max values only(couple_max)
        and return the couple_max'''
    couple_dict = Counter()
    for string in series:
        string = couple_creator_1(string)
        for actor in string:
            couple_dict[actor] += 1
            
    max_number = max(couple_dict.values())
    couple_max = {}              
    for couple in couple_dict:
        if couple_dict[couple] == max_number:
            couple_max[couple] = couple_dict[couple] 
            
    return couple_max

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [673]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'# +
# если ответили верно, можете добавить комментарий со значком "+"

In [674]:
# тут пишем ваш код для решения данного вопроса:
data.original_title[data.budget == data.budget.max()]

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

ВАРИАНТ 2

In [675]:
# можно добавлять разные варианты решения
data[data.budget.isin([data.budget.max()])][['original_title','imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [676]:
# ВАРИАНТ 3
data[data.budget == 3.800000e+08][['original_title','imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [677]:
# ВАРИАНТ 4
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,641683000


# 2. Какой из фильмов самый длительный (в минутах)?

In [678]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)' # +

In [679]:
# Вариант 1
data[data.runtime == data.runtime.max()][['original_title','imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


In [680]:
# Вариант 2
data[data.runtime == 214][['original_title','imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


In [681]:
# Вариант 3
data[data.runtime.isin([data.runtime.max()])][['original_title','imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [682]:
answers['3'] = 'Winnie the Pooh (tt1449283)' # +

In [683]:
print('Вариант 1')
display(data[data.runtime == data.runtime.min()][['original_title','imdb_id']])
print('Вариант 2')
display(data[data.runtime == 63][['original_title','imdb_id']])
print('Вариант 3')
data[data.runtime.isin([data.runtime.min()])][['original_title','imdb_id']]

Вариант 1


,original_title,imdb_id
768,Winnie the Pooh,tt1449283


Вариант 2


,original_title,imdb_id
768,Winnie the Pooh,tt1449283


Вариант 3


,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [684]:
answers['4'] = '110' # +

In [685]:
print('Вариант 1')
display(round(data.runtime.mean()))
print('Вариант 2')
data.describe()

Вариант 1


110

Вариант 2


,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


# 5. Каково медианное значение длительности фильмов? 

In [686]:
answers['5'] = '107' # +

In [687]:
print('Вариант 1')
display(round(data.runtime.median()))
print('Вариант 2')
data.describe()

Вариант 1


107

Вариант 2


,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [688]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [689]:
answers['6'] = 'Avatar (tt0499549)' # +

In [690]:
data[data.profit == data.profit.max()][['original_title','imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [691]:
answers['7'] = 'The Lone Ranger (tt1210819)' # +

In [692]:
data[data.profit == data.profit.min()][['original_title','imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [693]:
answers['8'] = '1478' # +

In [694]:
# вариант 1
len(data[data.profit > 0])

1478

In [695]:
# вариант 2
data.imdb_id[data.profit > 0].nunique()

1478

In [696]:
data[data.profit > 0].imdb_id.nunique()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [697]:
answers['9'] = 'The Dark Knight (tt0468569)' # +

In [698]:
# вариант 1
data[data.revenue == data[data.release_year == 2008]['revenue'].max()]\
[['original_title','imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


In [754]:
# вариант 2
data[data.revenue == data.groupby(['release_year'])\
     ['revenue'].max().loc[2008]][['original_title','imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [700]:
answers['10'] = 'The Lone Ranger (tt1210819)' # +

In [701]:
data[data.profit == data[(data.release_year >= 2012) &
                         (data.release_year <= 2014)].profit.min()]\
                        [['original_title','imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 11. Какого жанра фильмов больше всего?

In [702]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [703]:
 answers['11'] = 'Drama' # +

ВАРИАНТ 1

In [704]:
# вариант 1

# genres_s = data.genres.apply(str_split)
# genres_df = genres_s.reset_index()
# genres_df.explode('genres').genres.value_counts()

data.genres.apply(str_split).reset_index().explode('genres').\
genres.value_counts().index[0]

'Drama'

ВАРИАНТ 2

In [705]:
data.genres.str.split('|').explode().value_counts().index[0]

'Drama'

ВАРИАНТ 3

In [767]:
genre = list_creator(data.genres)
genre.most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [706]:
 answers['12'] = 'Drama' # +

In [707]:
data[data.profit > 0].genres.str.split('|').explode().value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [708]:
 answers['13'] = 'Peter Jackson' # +

In [709]:
changed_data = data.copy()
changed_data['director_split'] = changed_data.director.str.split('|')
c_d_director_exp = changed_data.explode('director_split')
sum_df = c_d_director_exp.groupby(['director_split']).sum()
sum_df[sum_df.revenue == sum_df.revenue.max()]

,budget,revenue,runtime,vote_average,release_year,profit
director_split,,,,,,
Peter Jackson,1288000000,6490593685,1355,58.1,16059,5202593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [710]:
 answers['14'] = 'Robert Rodriguez' # +

In [753]:
df = data.copy()
df['director_split'] = df.director.str.split('|')
df['genres_split'] = df.genres.str.split('|')
df_exp = df.explode('director_split')
df_exp = df_exp.explode('genres_split')
df_action = df_exp.loc[df_exp.genres_split == 'Action']
pivot = df_action.pivot_table(values = 'genres_split',
                              index = 'director_split',
                              aggfunc = 'count')
pivot[pivot.genres_split == pivot.genres_split.max()]

,genres_split
director_split,
Robert Rodriguez,9


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [712]:
 answers['15'] = 'Chris Hemsworth' # +

In [713]:
df = data.copy()
df['cast_split'] = df.cast.str.split('|')
df_exp = df.explode('cast_split')
df_2012 = df_exp[df_exp.release_year == 2012] 
df_2012_actor = df_2012.groupby('cast_split')
df_2012_actor.revenue.sum().sort_values(ascending = False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [714]:
 answers['16'] = 'Matt Damon' # +

In [715]:
df = data.copy()
df['cast_split'] = df.cast.str.split('|')
df_exp = df.explode('cast_split')
df_high_budget = df_exp[df_exp.budget > df_exp.budget.mean()]
df_high_budget_actor = df_high_budget.groupby('cast_split')
df_high_budget_actor.imdb_id.nunique().sort_values(ascending = False).index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [716]:
 answers['17'] = 'Action' # +

In [717]:
df = data.copy()
df['cast_split'] = df.cast.str.split('|')
df['genres_split'] = df.genres.str.split('|')
df_exp = df.explode('cast_split')
df_exp = df_exp.explode('genres_split')
df_NC = df_exp[df_exp.cast_split == 'Nicolas Cage']
df_NC.genres_split.value_counts().index[0]

'Action'

In [35]:
df = data.copy()
df[df.cast.str.contains('Nicolas Cage') == True].genres.str.split('|').explode().value_counts().index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [718]:
 answers['18'] = 'K-19: The Widowmaker (tt0267626)' # +

In [719]:
df = data.copy()
df['production_companies'] = df.production_companies.str.split('|')
df_exp = df.explode('production_companies')
df_PP = df_exp[df_exp.production_companies == 'Paramount Pictures']  
df_PP[df_PP.profit == df_PP.profit.min()][['original_title','imdb_id']]

,original_title,imdb_id
925,K-19: The Widowmaker,tt0267626


In [32]:
data2 = data.copy()
df = data2[data2.production_companies.str.contains('Paramount Pictures') == True]
df[df.profit == df.profit.min()][['original_title']]

,original_title
925,K-19: The Widowmaker


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [720]:
 answers['19'] = '2015' # +

In [721]:
data.groupby('release_year').revenue.sum().sort_values().index[-1]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [722]:
 answers['20'] = '2014' # +

In [752]:
df = data.copy()
df['production_companies'] = df.production_companies.str.split('|')
df_exp = df.explode('production_companies')
df_WB = df_exp[df_exp.production_companies.str.contains('Warner Bros',\
                                                        na = False)]
df_WB.groupby('release_year').profit.sum().sort_values().index[-1]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [724]:
 answers['21'] = '9' # +

In [725]:
data.release_date.dt.month.value_counts().index[0]

9

In [10]:
# вариант Михалила, проверка ошибки
data = pd.read_csv('movie_bd_v5.csv')
def return_month(release_date):
    lst_day_month_year = release_date.split("/")
    month_value = int(lst_day_month_year[0])
    switch = {
        1: "Январь",
        2: "Февраль",
        3: "Март",
        4: "Апрель",
        5: "Май",
        6: "Июнь",
        7: "Июль",
        8: "Август",
        9: "Сентябрь",
        10: "Октябрь",
        11: "Ноябрь",
#         12: "Январь" #  Ошибка Михаила
        12: "Декабрь"  #  мои правки
    }
    switch.setdefault(month_value, "Неверное значение")
    return switch[month_value]
data_2 = data.copy()
data_2["release_month"] = data_2["release_date"].apply(return_month)
# print(data_2.head(10))
# result_21 = 
data_2["release_month"].value_counts().index[0] 

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [726]:
 answers['22'] = '450' # +

In [727]:
data[data.release_date.dt.month == 6].imdb_id.nunique() + \
data[data.release_date.dt.month == 7].imdb_id.nunique() + \
data[data.release_date.dt.month == 8].imdb_id.nunique()

450

In [37]:
data[data.release_date.dt.month.isin([6,7,8])].imdb_id.nunique() 

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [728]:
 answers['23'] = 'Peter Jackson' # 

In [729]:
january = data[data.release_date.dt.month == 1].copy() 
february = data[data.release_date.dt.month == 2].copy()  
december = data[data.release_date.dt.month == 12].copy() 
winter = pd.concat([january,february,december])
winter['director'] = winter.director.str.split('|')
winter = winter.explode('director')
winter.director.value_counts().index[0]

'Peter Jackson'

In [40]:
data[data.release_date.dt.month.isin([1,2,12])].director.str.split('|').explode().value_counts().index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [730]:
 answers['24'] = 'Four By Two Productions' # 

In [731]:
df = data.copy()
df['production_companies'] = df.production_companies.str.split('|')
df = df.explode('production_companies')
df['len_title'] = df.original_title.str.len()
df.groupby('production_companies').len_title.mean().sort_values().index[-1]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [732]:
 answers['25'] = 'Midnight Picture Show' # 

In [733]:
df = data.copy()
df['production_companies'] = df.production_companies.str.split('|')
df = df.explode('production_companies')
df['overview'] = df.overview.apply(lambda string: len(string.split(' ')))
df.groupby('production_companies').overview.mean().sort_values().index[-1]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [734]:
 answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave' # 

In [735]:
# data.quantile(q = 0.99,numeric_only=True)

In [736]:
# data.vote_average.quantile(q = 0.99)

In [737]:
data[data.vote_average > data.vote_average.quantile(q = 0.99)].original_title

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

ВАРИАНТ 2

In [738]:
data[data.vote_average > data.vote_average.quantile(q = 0.99)]\
[['original_title','imdb_id']]

,original_title,imdb_id
9,Inside Out,tt2096673
34,Room,tt3170832
118,Interstellar,tt0816692
119,Guardians of the Galaxy,tt2015381
125,The Imitation Game,tt2084970
128,Gone Girl,tt2267998
138,The Grand Budapest Hotel,tt2278388
370,Inception,tt1375666
599,The Dark Knight,tt0468569
872,The Pianist,tt0253474


ВАРИАНТ 3


In [739]:
data.loc[data['vote_average'] > \
         data.quantile(0.99, numeric_only=True)['vote_average']]\
        [['original_title','vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [740]:
 answers['27'] = 'Daniel Radcliffe & Rupert Grint' # 

In [741]:
df = data[['cast']].copy()
df['cast'] = df.cast.apply(couple_creator)
df = df.explode('cast')
df.cast.value_counts().head(10)

Emma Watson & Rupert Grint            8
Daniel Radcliffe & Emma Watson        8
Daniel Radcliffe & Rupert Grint       8
Helena Bonham Carter & Johnny Depp    6
Ben Stiller & Owen Wilson             6
Paul Walker & Vin Diesel              5
Kristen Stewart & Robert Pattinson    5
Hugh Jackman & Ian McKellen           5
Adam Sandler & Kevin James            5
Robert Pattinson & Taylor Lautner     5
Name: cast, dtype: int64

ВАРИАНТ 2

In [742]:
df1 = data[['cast']].copy()
df1['cast'] = df1.cast.apply(couple_creator_1)
df1 = df1.explode('cast')
df1.cast.value_counts().head(10)

Emma Watson & Rupert Grint            8
Daniel Radcliffe & Rupert Grint       8
Daniel Radcliffe & Emma Watson        8
Helena Bonham Carter & Johnny Depp    6
Ben Stiller & Owen Wilson             6
Paul Walker & Vin Diesel              5
Adam Sandler & Kevin James            5
Kristen Stewart & Taylor Lautner      5
Robert Pattinson & Taylor Lautner     5
Kristen Stewart & Robert Pattinson    5
Name: cast, dtype: int64

ВАРИАНТ 3

In [743]:
df2 = data[['cast']].copy()
df2['cast'] = df2.cast.apply(couple_creator_2)
df2 = df2.explode('cast')
df2.cast.value_counts().head(10)

(Daniel Radcliffe, Rupert Grint)       8
(Daniel Radcliffe, Emma Watson)        8
(Rupert Grint, Emma Watson)            7
(Ben Stiller, Owen Wilson)             6
(Johnny Depp, Helena Bonham Carter)    6
(Hugh Jackman, Ian McKellen)           5
(Adam Sandler, Kevin James)            5
(Kristen Stewart, Taylor Lautner)      5
(Robert Pattinson, Taylor Lautner)     5
(Kristen Stewart, Robert Pattinson)    5
Name: cast, dtype: int64

При использовании couple_creator_2 результат отличается от couple_creator_1 иcouple_creator.

(Rupert Grint, Emma Watson) = 7 для couple_creator_2

Emma Watson & Rupert Grint = 8 для couple_creator_1 и для couple_creator

В couple_creator_2 используется itertools.combinations()


def couple_creator(cast_list):

    ''' The function create a list of actor's couples from a actor's list'''
    couple = []
    for actor in cast_list:
        for name in range(len(cast_list)):
            if actor > cast_list[name]:
                couple.append( cast_list[name] +' & ' + actor)
    return couple


def couple_creator_1(cast_list):

    ''' The function create a list of actor's couples from a actor's list'''
    couple = [actor +' & ' + cast_list[name] for actor in cast_list\
              for name in range(len(cast_list)) if cast_list[name] > actor]
    return couple


def couple_creator_2(cast_list):

    ''' The function create a list of actor's couples from a actor's list'''
    return list(combinations(cast_list, 2))
    
    
Так как на момент проверки все остальные строчки кода идентичны, решил проверить, какая функция врёт.

Позднее был изменён код функций и, соответственно, код - третьего варианта. Удалена строка кода df['cast'] = df.cast.str.split('|'), по причине переноса этого действия внутрь функций. Но тем не менее...

Получилось, что врёт couple_creator_2, т.е itertools.combinations(). 

Проверка ниже.

__Причина безобразия выявлена!__

itertools.combinations() коряво составляет пары, если его использовать построчно.


    ('Rupert Grint', 'Emma Watson') 7
    ('Emma Watson', 'Rupert Grint') 1

In [744]:
df_def_test = data[(data.cast.str.contains('Emma Watson')) &
      (data.cast.str.contains('Rupert Grint'))]
display(len(df_def_test))
display(df_def_test)

8

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
242,tt0417741,250000000,933959197,Harry Potter and the Half-Blood Prince,Daniel Radcliffe|Rupert Grint|Emma Watson|Tom ...,David Yates,Dark Secrets Revealed,"As Harry begins his sixth year at Hogwarts, he...",153,Adventure|Fantasy|Family,Warner Bros.|Heyday Films,2009-07-07,7.3,2009,683959197
374,tt0926084,250000000,954305868,Harry Potter and the Deathly Hallows: Part 1,Daniel Radcliffe|Emma Watson|Rupert Grint|Ralp...,David Yates,One Wayâ€¦ One Fateâ€¦ One Hero.,"Harry, Ron and Hermione walk away from their l...",146,Adventure|Fantasy|Family,Warner Bros.|Heyday Films,2010-10-17,7.4,2010,704305868
497,tt0241527,125000000,976475550,Harry Potter and the Philosopher's Stone,Daniel Radcliffe|Rupert Grint|Emma Watson|John...,Chris Columbus,Let the Magic Begin.,Harry Potter has lived under the stairs at his...,152,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films,2001-11-16,7.2,2001,851475550
722,tt1201607,125000000,1327817822,Harry Potter and the Deathly Hallows: Part 2,Daniel Radcliffe|Rupert Grint|Emma Watson|Alan...,David Yates,It all ends here.,"Harry, Ron and Hermione continue their quest t...",130,Adventure|Family|Fantasy,Warner Bros.|Heyday Films|Moving Picture Compa...,2011-07-07,7.7,2011,1202817822
864,tt0295297,100000000,876688482,Harry Potter and the Chamber of Secrets,Daniel Radcliffe|Rupert Grint|Emma Watson|Kenn...,Chris Columbus,Hogwarts is back in session.,"Ignoring threats to his life, Harry returns to...",161,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|MIRACL...,2002-11-13,7.2,2002,776688482
1312,tt0330373,150000000,895921036,Harry Potter and the Goblet of Fire,Daniel Radcliffe|Rupert Grint|Emma Watson|Ralp...,Mike Newell,Dark And Difficult Times Lie Ahead.,"Harry starts his fourth year at Hogwarts, comp...",157,Adventure|Fantasy|Family,Patalex IV Productions Limited|Warner Bros.|He...,2005-11-05,7.3,2005,745921036
1560,tt0304141,130000000,789804554,Harry Potter and the Prisoner of Azkaban,Daniel Radcliffe|Rupert Grint|Emma Watson|Gary...,Alfonso CuarÃ³n,Something wicked this way comes.,"Harry, Ron and Hermione return to Hogwarts for...",141,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|P of A...,2004-05-31,7.4,2004,659804554
1670,tt0373889,150000000,938212738,Harry Potter and the Order of the Phoenix,Daniel Radcliffe|Rupert Grint|Emma Watson|Mich...,David Yates,Evil Must Be Confronted.,Returning for his fifth year of study at Hogwa...,138,Adventure|Fantasy|Family|Mystery,Cool Music|Warner Bros.|Heyday Films|Harry Pot...,2007-06-28,7.2,2007,788212738


__Причина безобразия!__

In [745]:
df_def_test = data.loc[(data.cast.str.contains('Emma Watson')) &
      (data.cast.str.contains('Rupert Grint'))].copy()
df_def_test.cast = df_def_test.cast.apply(couple_creator_2)
all_couple_list = [couple_list for couple_list in df_def_test.cast ]
    
couple_dict = Counter()
for couple_in_film in all_couple_list:
    for couple in couple_in_film:
        couple_dict[couple] += 1        

print(f"('Rupert Grint', 'Emma Watson') \
      {couple_dict[('Rupert Grint', 'Emma Watson')]}")
print(f"('Emma Watson', 'Rupert Grint') \
      {couple_dict[('Emma Watson', 'Rupert Grint')]}")

('Rupert Grint', 'Emma Watson')       7
('Emma Watson', 'Rupert Grint')       1


ВАРИАНТ 4

In [746]:
# The function with use the itertools.combinations() 
dict_creator(data.cast)

{('Daniel Radcliffe', 'Rupert Grint'): 8,
 ('Daniel Radcliffe', 'Emma Watson'): 8}

In [747]:
# The function without use the itertools.combinations() 
dict_creator_1(data.cast)

{'Daniel Radcliffe & Rupert Grint': 8,
 'Daniel Radcliffe & Emma Watson': 8,
 'Emma Watson & Rupert Grint': 8}

ВАРИАНТ 5

In [748]:
couple_creator_3(data.cast)

[('Daniel Radcliffe & Rupert Grint', 8),
 ('Daniel Radcliffe & Emma Watson', 8),
 ('Emma Watson & Rupert Grint', 8),
 ('Ben Stiller & Owen Wilson', 6),
 ('Helena Bonham Carter & Johnny Depp', 6),
 ('Paul Walker & Vin Diesel', 5),
 ('Adam Sandler & Kevin James', 5),
 ('Hugh Jackman & Ian McKellen', 5),
 ('Kristen Stewart & Robert Pattinson', 5),
 ('Kristen Stewart & Taylor Lautner', 5)]

In [749]:
# он же
Counter([couple for couple_list in data.cast.apply(couple_creator_1)\
                    for couple in couple_list]).most_common(10)

[('Daniel Radcliffe & Rupert Grint', 8),
 ('Daniel Radcliffe & Emma Watson', 8),
 ('Emma Watson & Rupert Grint', 8),
 ('Ben Stiller & Owen Wilson', 6),
 ('Helena Bonham Carter & Johnny Depp', 6),
 ('Paul Walker & Vin Diesel', 5),
 ('Adam Sandler & Kevin James', 5),
 ('Hugh Jackman & Ian McKellen', 5),
 ('Kristen Stewart & Robert Pattinson', 5),
 ('Kristen Stewart & Taylor Lautner', 5)]

# Submission

In [750]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': '9',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [751]:
# и убедиться что ни чего не пропустил)
len(answers)

27